Import the necessary packages for the SafetySettings of the Gemini

In [1]:
from google.cloud import aiplatform
from vertexai.preview.generative_models import GenerativeModel, ChatSession, HarmCategory, SafetySetting
from google.api_core.exceptions import InvalidArgument, PermissionDenied
import os

Configure the GCP Project

In [2]:
project = "qwiklabs-gcp-02-4c9c7fb5e8ec"
location = "us-central1"
aiplatform.init(project=project, location=location)

Initialize the threshold of the Safety Settings of the gemini

In [3]:
safety_settings = [
    SafetySetting(category=HarmCategory.HARM_CATEGORY_HARASSMENT, threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category=HarmCategory.HARM_CATEGORY_HATE_SPEECH, threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT, threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT, threshold="BLOCK_LOW_AND_ABOVE"),
]

Generate the Gemini Modal with the Safety Settings configured

In [4]:
model = GenerativeModel(
    "gemini-2.0-flash",
    safety_settings=safety_settings,
)

In [5]:
chat = model.start_chat()

Implement prompt filtering and Validate model responses for safety

In [6]:
def is_prompt_safe(prompt: str) -> bool:
    """
    Check whether the user's prompt is considered safe by Gemini's safety filters.
    """
    try:
        response = model.generate_content(
            prompt,
            safety_settings=safety_settings,
            stream=False
        )

        feedback = response.prompt_feedback
        if feedback and feedback.block_reason:
            print(f"❌ Prompt blocked due to: {feedback.block_reason}")
            return False

        return True
    except Exception as e:
        print(f"⚠️ Error during prompt validation: {e}")
        return False


In [7]:
def is_response_safe(response) -> bool:
    """
    Check whether the response is considered safe by Gemini's safety filters.
    """
    try:
        feedback = response.prompt_feedback
        if feedback and feedback.block_reason:
            print(f"❌ Response blocked due to: {feedback.block_reason}")
            return False
        return True
    except Exception as e:
        print(f"⚠️ Error during response validation: {e}")
        return False


In [8]:
def safe_chat_with_user(prompt: str):
    """
    Chat with the user, filtering unsafe prompts and responses.
    """
    if not is_prompt_safe(prompt):
        print("Gemini: Sorry, your input violates safety guidelines.")
        return

    response = chat.send_message(prompt, safety_settings=safety_settings)

    if not is_response_safe(response):
        print("Gemini: I generated a response, but it was blocked for safety reasons.")
        return

    print("Gemini:", response.text.strip())

# Run the chatbot
print("🤖 Gemini Chatbot (type 'exit' or 'quit' to leave)")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit","bye"]:
        print("👋 Bye!")
        break
    safe_chat_with_user(user_input)

🤖 Gemini Chatbot (type 'exit' or 'quit' to leave)
You: hi. I am an aspiring engineer. I would like some tips on recent technology facts
Gemini: Okay, great! Aspiring engineers need to stay informed about recent tech developments. Here's a breakdown of recent technology facts and tips on how to stay updated:

**Key Areas to Focus On:**

*   **Artificial Intelligence (AI) & Machine Learning (ML):**
    *   **Large Language Models (LLMs):** (Think GPT-4, Llama 2, Gemini)
        *   **Fact:** LLMs are rapidly evolving in terms of reasoning, coding capabilities, and multi-modality (handling text, images, audio, etc.).
        *   **Fact:**  Open-source LLMs are becoming increasingly powerful, closing the gap with proprietary models.
        *   **Example:**  The emergence of agents built on top of LLMs that can autonomously perform tasks by interacting with real-world tools and APIs.
    *   **Generative AI:**
        *   **Fact:** Generative AI is now being used to create not just images 